# Cyton Signal Acquisition

This notebook demonstrates how to acquire data from the Cyton board using the brainflow library (and a custom module). The Cyton board is a 8-channel EEG board that can be used to acquire EEG data.

**NOTE:** If you want to use the custom module, you will need to have the file `brainflow_stream.py` in the same directory as this notebook (or other scripts that use it).

## Installation

Make sure you have the provided `neurohack` python environment installed.

Or you can install the following packages:

```bash
conda install brainflow
conda install scipy
conda install matplotlib
conda install pyserial
```

In [1]:
import scipy
import numpy as np
import matplotlib.pyplot as plt
import time

import brainflow
from brainflow.board_shim import BoardShim, BrainFlowInputParams, BrainFlowError, BoardIds


# Import the custom module.
from brainflow_stream import BrainFlowBoardSetup

# Connecting to the cyton board

## Create a BrainFlowBoard object

The BrainFlowBoardSetup object is a custom wrapper to the Brainflow BoardShim object. There is some added utility to the custom wrapper, such as the ability to automatically detect the serial port of the Cyton board.

All attributes of the BrainFlowBoardSetup are: - Also see the `brainflow_stream.py` module for more details.
- name (str): A user-friendly name or identifier for the board setup instance. (Useful when connecting to 2+ boards simultaneously)
- board_id (int): The ID of the BrainFlow board to use.
- serial_port (str): The serial port to which the BrainFlow board is connected.
- master_board (int): (Optional, ONLY if using playback or synthetic boards) The ID of the master board.
- params (BrainFlowInputParams): (Optional) Instance of BrainFlowInputParams representing the board's input parameters.
- board (BoardShim): Instance of BoardShim representing the active board.
- session_prepared (bool): Flag indicating if the session has been prepared.
- streaming (bool): Flag indicating if the board is actively streaming data.
- eeg_channels (list): List of EEG channel indices for the board (empty if not applicable).
- sampling_rate (int): Sampling rate of the board.

In [ ]:
board_id = BoardIds.CYTON_BOARD.value # Set the board_id to match the Cyton board

# Lets quickly take a look at the specifications of the Cyton board
for item1, item2 in BoardShim.get_board_descr(board_id).items():
    print(f"{item1}: {item2}")

In [ ]:
cyton_board = BrainFlowBoardSetup(
                                board_id = board_id,
                                name = 'Board_1', # Optional name for the board. This is useful if you have multiple boards connected and want to distinguish between them.
                                serial_port = None # If the serial port is not specified, it will try to auto-detect the board. If this fails, you will have to assign the correct serial port. See https://docs.openbci.com/GettingStarted/Boards/CytonGS/ 
                                ) 

cyton_board.setup() # This will establish a connection to the board and start streaming data.

Once the board is connected and streaming, we can call some utility functions to get some information about the board.

In [ ]:
board_info = cyton_board.get_board_info() # Retrieves the EEG channel and sampling rate of the board.
print(f"Board info: {board_info}")

board_srate = cyton_board.get_sampling_rate() # Retrieves the sampling rate of the board.
print(f"Board sampling rate: {board_srate}")

We can also look at the specifications of the board.

## Get Data

Once connected, the board starts streaming data into the computers' buffer where it is stored temporarily. The default buffer size is 450000 (at 250Hz sampling rate, this is 30 minutes). We can then pull data from the buffer and process it.

There are **Two ways** to pull data from the board/buffer.
1. `get_board_data()` -> Retrieves and clears ALL samples from the buffer. 
2. `get_current_board_data(num_samples)` -> Retrieves the last `num_samples` samples without clearing them from the buffer.

For more information on the buffer and behind-the-scenes see the [brainflow documentation](https://brainflow.readthedocs.io/en/stable/UserAPI.html#brainflow-board-shim).

In [ ]:
time.sleep(5) # Wait for 5 seconds to allow the board to build up some samples into the buffer

raw_data_500 = cyton_board.get_current_board_data(num_samples = 500) # Get the latest 500 samples from the buffer
print(f"raw_data_1000 shape: {raw_data_500.shape}")

raw_data_all = cyton_board.get_board_data() 
print(f"raw_data_all shape: {raw_data_all.shape}")

As we can see here, each method returns a tuple of two arrays in shape **(n_channels, n_samples)**

Since the cyton_board only has 8 eeg channels, the other channels contain data on other sensors like the accelerometer, gyroscope, etc.

For the Cyton boards this is the channel mapping:
- {'accel_channels': [9, 10, 11], 
- 'analog_channels': [19, 20, 21], 
- 'eeg_channels': [1, 2, 3, 4, 5, 6, 7, 8], 
- 'eeg_names': 'Fp1,Fp2,C3,C4,P7,P8,O1,O2', 
- 'marker_channel': 23, 
- 'other_channels': [12, 13, 14, 15, 16, 17, 18], 
- 'package_num_channel': 0, 
- 'sampling_rate': 250, 
- 'timestamp_channel': 22}

So, lets slice out only the EEG channels from our raw data

In [ ]:
eeg_data = raw_data_500[1:9, :] # Get the EEG data from the first 8 channels
print(f"eeg_data shape: {eeg_data.shape}")

Great, now lets visualize the data using matplotlib

In [ ]:
num_channels = eeg_data.shape[0]
num_samples = eeg_data.shape[1]

# Create a figure and a set of subplots
fig, axes = plt.subplots(num_channels, 1, figsize=(10, 2 * num_channels), sharex=True, sharey=True)

# Plot each channel
for i in range(num_channels):
    axes[i].plot(eeg_data[i, :])
    axes[i].set_title(f'Channel {i+1}')
    axes[i].set_ylabel('Amplitude (µV)')

axes[-1].set_xlabel('Samples')

plt.show()

# Basic data processing (DC offset)

If you look at the above graph of the raw data, the X-axis looks off - those values are far too large! Let's normalize the data so that it's centered around 0. 

In [ ]:
# To do this we can subtract the mean of the data from the data itself. This will center the data around zero.
eeg_data_dc_removed = eeg_data - np.mean(eeg_data, axis=1, keepdims=True)

# While we're at it, lets make a small function that performs all of our minimal processing this since we'll have to do it every time we pull data from the board.
def remove_dc_offset(data):
    return data[1:9, :] - np.mean(data[1:9, :], axis=1, keepdims=True)

In [ ]:
# Now let's plot the results
num_channels = eeg_data_dc_removed.shape[0]
num_samples = eeg_data_dc_removed.shape[1]

fig, axes = plt.subplots(num_channels, 1, figsize=(10, 2 * num_channels), sharex=True, sharey=True)

for i in range(num_channels):
    axes[i].plot(eeg_data_dc_removed[i, :])
    axes[i].set_title(f'Channel {i+1}')
    axes[i].set_ylabel('Amplitude (µV)')

axes[-1].set_xlabel('Samples')

plt.show()

That looks much better!

# Event Markers

Event markers are very useful for BCI systems. They are used to mark specific times in the EEG stream where an event occured. This could be a stimulus presentation, a button press, etc. This allows us to segment the data into epochs or windows of EEG data around when the event and analyze the data in a more meaningful way.


We can add markers to the data stream by calling `insert_marker()`. The marker itself has to be a number, and will have to be decoded later on. (i.e., what does the marker number 5 mean?)

Lets make a small loop which inserts a marker every 0.5 seconds for 5 seconds.

In [ ]:
for _ in range(10):  # 5 seconds / 0.5 seconds = 10 iterations
    cyton_board.insert_marker(marker=10,
                                verbose=True # You can set this to False if you don't want to print the marker value
                                )
    time.sleep(0.5)
    
# Then, lets pull the last 5 seconds of data (250 samples per second * 5 seconds = 1250 samples)
raw_data_1250 = cyton_board.get_current_board_data(num_samples = 1250)

Now, lets do our minimal processing by taking out the marker channel, and then the EEG channels normalizing the raw data.

In [ ]:
# Extract marker channel (assuming it's at index 23)
marker_data = raw_data_1250[23, :]  # Single row, all time points

# Lets use the function we made earlier to extract eeg channels and normalize the data!
eeg_data_dc_offset_removed = remove_dc_offset(eeg_data)

Now, lets visualize the EEG data, with a vertical line for each marker.

In [ ]:
# Get number of samples
num_samples = eeg_data_dc_offset_removed.shape[1]

# Find marker event indices
event_indices = np.where(marker_data == 10)[0] # Find the samples where the marker is 10

# Plot EEG data with markers
plt.figure(figsize=(12, 6))
offset = 50  # Spacing between channels for easier visualization

for i in range(eeg_data_dc_offset_removed.shape[0]):
    plt.plot(eeg_data_dc_offset_removed[i] + i * offset, label=f'Ch {i+1}')

# Add vertical lines for markers
for event in event_indices:
    plt.axvline(event, color='r', linestyle='--', linewidth=1)

plt.xlabel("Samples")
plt.ylabel("EEG Channels")
plt.legend(loc="upper right")
plt.show()

# Converting cyton data to an MNE object
**[MNE Python](https://mne.tools/stable/index.html)** is a powerful python library for EEG data analysis. It has a lot of built-in functions for data processing, visualization, and analysis.

In this section I will quickly demonstrate how you can take the raw data from the Cyton board and convert it into an MNE Raw object, which can then be used with all the MNE functions.

In [ ]:
# First, lets take a large amount (30 seconds) to make the conversion worthwhile

time.sleep(30) # We have to wait for the buffer to fill up with data before we can pull it.

raw_data_7500 = cyton_board.get_current_board_data(num_samples = 7500)

# Then lets perform our minimal processing on the data normalize the data
eeg_data_cleaned = remove_dc_offset(raw_data_7500)

In [ ]:
# Then lets perform our minimal processing on the data normalize the data
eeg_data_cleaned = remove_dc_offset(raw_data_7500)

In [ ]:
import mne

# Get channel names -> Can also define them manually
ch_names = cyton_board.get_eeg_names(board_id)

# Define channel types (MNE needs this)
ch_types = ['eeg'] * len(ch_names)

sampling_rate = cyton_board.get_sampling_rate()

# Create MNE info structure
info = mne.create_info(ch_names=ch_names, sfreq=sampling_rate, ch_types=ch_types)


In [ ]:
# For MNE to work properly, we need to convert the data to Volts
eeg_data_cleaned = eeg_data_cleaned * 1e-6 # Multiply the EEG data by 1e-6 to convert to Volts

# Create the RawArray object
raw = mne.io.RawArray(eeg_data_cleaned, info)

# Plot to verify
raw.plot()

Finally, lets stop streaming from the board.

In [ ]:
cyton_board.stop()

# Resources

- [Brainflow Python Documentation](https://brainflow.readthedocs.io/en/stable/Examples.html#python)
- [MNE Python](https://mne.tools/stable/index.html)